In [0]:
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'https://drive.google.com/open?id=15kfFNM_J7rKZUypniYqBM-smiX7qJnIN'

In [0]:
fluff, id = link.split('=')
print (id)

15kfFNM_J7rKZUypniYqBM-smiX7qJnIN


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('column_3c_weka.csv')  
data = pd.read_csv('column_3c_weka.csv')

In [0]:
data.shape

(310, 7)

In [0]:
data.head
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 7 columns):
pelvic_incidence            310 non-null float64
pelvic_tilt                 310 non-null float64
lumbar_lordosis_angle       310 non-null float64
sacral_slope                310 non-null float64
pelvic_radius               310 non-null float64
degree_spondylolisthesis    310 non-null float64
class                       310 non-null object
dtypes: float64(6), object(1)
memory usage: 17.1+ KB


In [0]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
data['class']=lb.fit_transform(data['class'])

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 7 columns):
pelvic_incidence            310 non-null float64
pelvic_tilt                 310 non-null float64
lumbar_lordosis_angle       310 non-null float64
sacral_slope                310 non-null float64
pelvic_radius               310 non-null float64
degree_spondylolisthesis    310 non-null float64
class                       310 non-null int64
dtypes: float64(6), int64(1)
memory usage: 17.1 KB


In [0]:
x=data.drop('class',axis=1)
y=data['class']

In [0]:
data.isnull().sum()

pelvic_incidence            0
pelvic_tilt                 0
lumbar_lordosis_angle       0
sacral_slope                0
pelvic_radius               0
degree_spondylolisthesis    0
class                       0
dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size =0.2 ,stratify=y,random_state=42)


In [0]:
mlpClassifier = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(10, 5), random_state=1)

In [0]:
mlpClassifier.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 5), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
score = mlpClassifier.score(X_test, y_test)
print(score)

0.8387096774193549


In [0]:
import tensorflow as tf
from tensorflow import keras

In [0]:
tf.set_random_seed(42)
np.random.seed(42)

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [0]:
sc.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [0]:
model_MLP = keras.models.Sequential()

model_MLP.add(keras.layers.Dense(units=21, activation='relu',input_shape= X_train.shape[1:]))

model_MLP.add(keras.layers.Dense(units=7, activation='softmax'))

model_MLP.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 21)                147       
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 154       
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [0]:
model_MLP.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [0]:
model_MLP.fit(x=X_train_std, y=y_train, validation_split=0.1, epochs=5,batch_size=16)

Train on 223 samples, validate on 25 samples
Epoch 1/5
223/223 [==============================] - 0s 1ms/sample - loss: 1.6805 - acc: 0.4439 - val_loss: 1.6284 - val_acc: 0.4400
Epoch 2/5
223/223 [==============================] - 0s 131us/sample - loss: 1.5425 - acc: 0.5516 - val_loss: 1.4926 - val_acc: 0.6400
Epoch 3/5
223/223 [==============================] - 0s 114us/sample - loss: 1.4156 - acc: 0.6413 - val_loss: 1.3663 - val_acc: 0.6800
Epoch 4/5
223/223 [==============================] - 0s 109us/sample - loss: 1.3049 - acc: 0.6682 - val_loss: 1.2513 - val_acc: 0.6800
Epoch 5/5
223/223 [==============================] - 0s 135us/sample - loss: 1.2030 - acc: 0.7040 - val_loss: 1.1523 - val_acc: 0.7200


In [0]:
test_loss, test_accuracy = model_MLP.evaluate(x=X_test_std, y=y_test)

62/62 [==============================] - 0s 145us/sample - loss: 1.2003 - acc: 0.6452


In [0]:
print(test_loss,test_accuracy)

1.2002646038609166 0.6451613
